In [44]:
from __future__ import print_function

"""
Tutorial: A reference implementation of non-hermitian cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

$\langle \mu \rangle$ represents the dipole expectation value computed at the Hartree-Fock level.
The dipole operator has the form (in first quantization):
$$ \hat{\mu} = \sum_{i}^{N_e} \mu(x_i) + \sum_{A}^{N_N} \mu_{nuc}(x_A) $$
where $\mu(x_i)$ depends on electronic coordinates and $\mu_{nuc}(x_A)$ depends on nuclear coordinates.

We will check to make sure we can compute the RHF dipole expectation
value using the dipole integrals below.

In [61]:
# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time

# ==> Set Basic Psi4 Options <==

# Memory specifications
psi4.set_memory(int(2e9))
numpy_memory = 2

# Output options
psi4.core.set_output_file('output.dat', False)

Define the molecule!

In [62]:
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e` and `scf_wfn`. We also check the memory requirements for computation:

In [63]:
# Get the SCF wavefunction & energies
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()
nmo = wfn.nmo()



We first obtain orbital information from our wavefunction. We also create an instance of MintsHelper to help build our molecular integrals:

In [65]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc   = wfn.nalpha()

# total number of orbitals
nmo     = wfn.nmo()

# number of virtual orbitals
nvirt   = nmo - ndocc

# orbital energies
eps     = np.asarray(wfn.epsilon_a())

# occupied orbitals:
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals:
Cv = wfn.Ca_subset("AO", "VIR")

# grab all transformation vectors and store to a numpy array!
C = np.asarray(wfn.Ca())

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

S = np.asarray(mints.ao_overlap())

# Get nbf and ndocc for closed shell molecules
nbf = S.shape[0]
ndocc = wfn.nalpha()

print("\nNumber of occupied orbitals: %d" % ndocc)
print("Number of basis functions: %d" % nbf)

# Run a quick check to make sure everything will fit into memory
I_Size = (nbf ** 4) * 8.0e-9
print("\nSize of the ERI tensor will be %4.2f GB." % I_Size)

# Estimate memory usage
memory_footprint = I_Size * 1.5
if I_Size > numpy_memory:
    psi4.core.clean()
    raise Exception(
        "Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                    limit of %4.2f GB."
        % (memory_footprint, numpy_memory)
    )

# Compute required quantities for SCF
print("V")
V = np.asarray(mints.ao_potential())
print(V)
T = np.asarray(mints.ao_kinetic())
I = np.asarray(mints.ao_eri())



Number of occupied orbitals: 5
Number of basis functions: 7

Size of the ERI tensor will be 0.00 GB.
V
[[-6.15805953e+01 -7.41082186e+00 -1.44738338e-02  0.00000000e+00
  -2.16840434e-18 -1.23168563e+00 -1.23168563e+00]
 [-7.41082186e+00 -1.00090712e+01 -1.76890224e-01  0.00000000e+00
  -4.16333634e-17 -2.97722676e+00 -2.97722676e+00]
 [-1.44738338e-02 -1.76890224e-01 -9.94404288e+00  0.00000000e+00
  -6.93889390e-18 -1.47178808e+00 -1.47178808e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.87587601e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.16840434e-18 -4.16333634e-17 -6.93889390e-18  0.00000000e+00
  -9.98755043e+00  1.82224083e+00 -1.82224083e+00]
 [-1.23168563e+00 -2.97722676e+00 -1.47178808e+00  0.00000000e+00
   1.82224083e+00 -5.30020268e+00 -1.06716576e+00]
 [-1.23168563e+00 -2.97722676e+00 -1.47178808e+00  0.00000000e+00
  -1.82224083e+00 -1.06716576e+00 -5.30020268e+00]]


We can transform the dipole integrals from the AO to the MO basis in the following way:
\begin{equation}
{\bf \mu}^{\xi} = {\bf C}^T {\bf \mu}^{\xi}_{ao} {\bf C},
\end{equation}
where ${\bf \mu}^{\xi}_{ao}$ represents the $\xi$ component of the dipole integrals in the AO basis and ${\bf C}$
represents the matrix of transformation vectors that go from AOs to MOs.  Note here 
$\xi$ can be $x$, $y$, or $z$.  At first pass, we will consider only the $z$ component, which means that 
the molecule will only couple to photons polarized along the $z$ axis.

In [66]:
Ex = 0
Ey = 1e-5
Ez = 1e-3
lam = np.array([Ex, Ey, Ez])
print(lam[1])

1e-05


In [67]:
# number of doubly occupied orbitals
ndocc = wfn.nalpha()

# Extra terms for Pauli-Fierz Hamiltonian
# nuclear dipole
mu_nuc_x = mol.nuclear_dipole()[0]
mu_nuc_y = mol.nuclear_dipole()[1]
mu_nuc_z = mol.nuclear_dipole()[2]

# dipole arrays in AO basis
mu_x_ao = np.asarray(mints.ao_dipole()[0])
mu_y_ao = np.asarray(mints.ao_dipole()[1])
mu_z_ao = np.asarray(mints.ao_dipole()[2])

print(mu_x_ao)
# transform dipole array to MO basis from ordinary RHF (no photon)
mu_x = np.dot(C.T, mu_x_ao).dot(C)
mu_y = np.dot(C.T, mu_y_ao).dot(C)
mu_z = np.dot(C.T, mu_z_ao).dot(C)

# compute components of electronic dipole moment <mu> from ordinary RHF (no photon)
mu_exp_x = 0.0
mu_exp_y = 0.0
mu_exp_z = 0.0
for i in range(0, ndocc):
    # double because this is only alpha terms!
    mu_exp_x += 2 * mu_x[i, i]
    mu_exp_y += 2 * mu_y[i, i]
    mu_exp_z += 2 * mu_z[i, i]

# We need to carry around the electric field dotted into the nuclear dipole moment
# and the electric field dotted into the RHF electronic dipole expectation value...
# so let's compute them here!

# lambda . mu_nuc
l_dot_mu_nuc = lam[0] * mu_nuc_x + lam[1] * mu_nuc_y + lam[2] * mu_nuc_z
l_dot_mu_el = lam[0] * mu_exp_x + lam[1] * mu_exp_y + lam[2] * mu_exp_z
l_dot_mu = l_dot_mu_nuc + l_dot_mu_el

# dipole constants to add to E_RHF
l_dot_mu_constant = l_dot_mu_nuc ** 2 + l_dot_mu_nuc * l_dot_mu + l_dot_mu ** 2

# quadrupole arrays
Q_xx = np.asarray(mints.ao_quadrupole()[0])
Q_xy = np.asarray(mints.ao_quadrupole()[1])
Q_xz = np.asarray(mints.ao_quadrupole()[2])
Q_yy = np.asarray(mints.ao_quadrupole()[3])
Q_yz = np.asarray(mints.ao_quadrupole()[4])
Q_zz = np.asarray(mints.ao_quadrupole()[5])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.07919296e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.41172844e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.08620836e-18
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.07919296e-02 -6.41172844e-01  9.08620836e-18  0.00000000e+00
   0.00000000e+00 -2.48967926e-01 -2.48967926e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.48967926e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.48967926e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [69]:
# ordinary H_core
H_0 = T + V

# Pauli-Fierz 1-e quadrupole terms
Q_PF = lam[0] * lam[0] * Q_xx
Q_PF += lam[1] * lam[1] * Q_yy
Q_PF += lam[2] * lam[2] * Q_zz
Q_PF += 2 * lam[0] * lam[1] * Q_xy
Q_PF += 2 * lam[0] * lam[2] * Q_xz
Q_PF += 2 * lam[1] * lam[2] * Q_yz

# Pauli-Fierz 1-e dipole terms scaled by l . <mu>
d_PF = 2 * l_dot_mu * lam[0] * mu_x
d_PF += 2 * l_dot_mu * lam[1] * mu_y
d_PF += 2 * l_dot_mu * lam[2] * mu_z
H = H_0 + Q_PF + d_PF

# Orthogonalizer A = S^(-1/2) using Psi4's matrix power.
A = mints.ao_overlap()
A.power(-0.5, 1.0e-16)
A = np.asarray(A)

# Calculate initial core guess: [Szabo:1996] pp. 145
Hp = A.dot(H).dot(A)  # Eqn. 3.177
e, C2 = np.linalg.eigh(Hp)  # Solving Eqn. 1.178
C = A.dot(C2)  # Back transform, Eqn. 3.174
Cocc = C[:, :ndocc]

D = np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

#print("\nTotal time taken for setup: %.3f seconds" % (time.time() - t))

print("\nStart SCF iterations:\n")
t = time.time()
E = 0.0
Enuc = mol.nuclear_repulsion_energy()
Eold = 0.0
Dold = np.zeros_like(D)

E_1el = np.einsum("pq,pq->", H + H, D) + Enuc + l_dot_mu_constant
print("One-electron energy = %4.16f" % E_1el)




Start SCF iterations:

One-electron energy = -117.8397131756816520


In [72]:
# Set defaults
maxiter = 40
E_conv = 1.0e-6
D_conv = 1.0e-3

for SCF_ITER in range(1, maxiter + 1):

    # Build fock matrix: [Szabo:1996] Eqn. 3.154, pp. 141
    J = np.einsum("pqrs,rs->pq", I, D)
    K = np.einsum("prqs,rs->pq", I, D)
    mu_x
    # Pauli-Fierz dipole-dipole matrices
    M_xx = np.einsum("pq,rs,rs->pq", lam[0] * mu_x, lam[0] * mu_x, D)
    M_yy = np.einsum("pq,rs,rs->pq", lam[1] * mu_y, lam[1] * mu_y, D)
    M_zz = np.einsum("pq,rs,rs->pq", lam[2] * mu_z, lam[2] * mu_z, D)

    M_xy = np.einsum("pq,rs,rs->pq", lam[0] * mu_x, lam[1] * mu_y, D)
    M_xz = np.einsum("pq,rs,rs->pq", lam[0] * mu_x, lam[2] * mu_z, D)
    M_yz = np.einsum("pq,rs,rs->pq", lam[1] * mu_y, lam[2] * mu_z, D)

    # Pauli-Fierz dipole-dipole "exchange" terms
    N_xx = np.einsum("pr,qs,rs->pq", lam[0] * mu_x, lam[0] * mu_x, D)
    N_yy = np.einsum("pr,qs,rs->pq", lam[1] * mu_y, lam[1] * mu_y, D)
    N_zz = np.einsum("pr,qs,rs->pq", lam[2] * mu_z, lam[2] * mu_z, D)

    N_xy = np.einsum("pr,qs,rs->pq", lam[0] * mu_x, lam[1] * mu_y, D)
    N_xz = np.einsum("pr,qs,rs->pq", lam[0] * mu_x, lam[2] * mu_z, D)
    N_yz = np.einsum("pr,qs,rs->pq", lam[1] * mu_y, lam[2] * mu_z, D)

    # Build fock matrix: [Szabo:1996] Eqn. 3.154, pp. 141 +
    # Pauli-Fierz terms
    F = H + J * 2 - K
    F += lam[0] ** 2 * M_xx
    F += lam[1] ** 2 * M_yy
    F += lam[2] ** 2 * M_zz

    F += 2 * lam[0] * lam[1] * M_xy
    F += 2 * lam[0] * lam[2] * M_xz
    F += 2 * lam[1] * lam[2] * M_yz

    F -= 0.5 * lam[0] ** 2 * N_xx
    F -= 0.5 * lam[1] ** 2 * N_yy
    F -= 0.5 * lam[2] ** 2 * N_zz

    F -= lam[0] * lam[1] * N_xy
    F -= lam[0] * lam[2] * N_xz
    F -= lam[1] * lam[2] * N_yz
    
    
    diis_e = np.einsum("ij,jk,kl->il", F, D, S) - np.einsum("ij,jk,kl->il", S, D, F)
    diis_e = A.dot(diis_e).dot(A)
    dRMS = np.mean(diis_e ** 2) ** 0.5

    # SCF energy and update: [Szabo:1996], Eqn. 3.184, pp. 150
    SCF_E = np.einsum("pq,pq->", F + H, D) + Enuc

    print(
        "SCF Iteration %3d: Energy = %4.16f   dE = % 1.5E   dRMS = %1.5E"
        % (SCF_ITER, SCF_E, (SCF_E - Eold), dRMS)
    )
    if (abs(SCF_E - Eold) < E_conv) and (dRMS < D_conv):
        break

    Eold = SCF_E
    Dold = D

    # Diagonalize Fock matrix: [Szabo:1996] pp. 145
    Fp = A.dot(F).dot(A)  # Eqn. 3.177
    e, C2 = np.linalg.eigh(Fp)  # Solving Eqn. 1.178
    C = A.dot(C2)  # Back transform, Eqn. 3.174
    Cocc = C[:, :ndocc]
    D = np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

    if SCF_ITER == maxiter:
        psi4.core.clean()
        raise Exception("Maximum number of SCF cycles exceeded.")

print("Total time for SCF iterations: %.3f seconds \n" % (time.time() - t))

print("Final SCF energy: %.8f hartree" % SCF_E)

psi4.compare_values(scf_e, SCF_E, 6, "SCF Energy")

SCF Iteration   1: Energy = -73.2858012392397313   dE = -7.32858E+01   dRMS = 1.20729E-01
SCF Iteration   2: Energy = -74.8281308880580980   dE = -1.54233E+00   dRMS = 5.08294E-02
SCF Iteration   3: Energy = -74.9354951333080805   dE = -1.07364E-01   dRMS = 1.32043E-02
SCF Iteration   4: Energy = -74.9414850058865056   dE = -5.98987E-03   dRMS = 2.76691E-03
SCF Iteration   5: Energy = -74.9419792725741729   dE = -4.94267E-04   dRMS = 1.16165E-03
SCF Iteration   6: Energy = -74.9420633404597254   dE = -8.40679E-05   dRMS = 4.96538E-04
SCF Iteration   7: Energy = -74.9420816988153717   dE = -1.83584E-05   dRMS = 2.41745E-04
SCF Iteration   8: Energy = -74.9420859274760289   dE = -4.22866E-06   dRMS = 1.15543E-04
SCF Iteration   9: Energy = -74.9420869100164566   dE = -9.82540E-07   dRMS = 5.58584E-05
Total time for SCF iterations: 30.644 seconds 

Final SCF energy: -74.94208691 hartree
    SCF Energy............................................................................FAILED


TestComparisonError: 	SCF Energy: computed value (-74.94208691) does not match (-74.94207990) to atol=1e-06 by difference (-0.00000701).

In [71]:
# Get the z-component of the RHF dipole moment from psi4.
# this will be in debye, and we will want to convert it into
# au to match those computed from dipole integrals
muz_psi4_debye = psi4.core.variable('SCF DIPOLE Z')
#qz_psi4_debye = psi4.core.variable('SCF QUADRUPOLE Z')
muz_psi4_au = muz_psi4_debye/2.54174623

# Get the nuclear dipole moment from psi4
nuc_dipole = mol.nuclear_dipole()



# now try to compute the dipole expectation value from the 
# dipole integrals
ed_z = 0.
for i in range(0, ndocc):
    ed_z += mu_z[i,i]

muz_computed = 2 * ed_z + nuc_dipole[2]    
print("Computed from integrals:",muz_computed)
print("Psi4:                   ",muz_psi4_au)
print("These dipole moments are close:",np.isclose(muz_computed,muz_psi4_au))



Computed from integrals: 0.6035212529279848
Psi4:                    0.6035213056155281
These dipole moments are close: True


<ipython-input-71-70fb2822b968>:4: FutureWarning: Using scalar QCVariable `SCF DIPOLE Z` [D] instead of array `SCF DIPOLE` [e a0] is deprecated, and in 1.5 it will stop working

  muz_psi4_debye = psi4.core.variable('SCF DIPOLE Z')
